In [22]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils import two_point_calibration, average_bottle, plot_sensor_measurement


DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")
    
df_p = pl.scan_parquet(os.path.join(PICARRO_DATA_DIRECTORY, "picarro.parquet")) 

# ICOS Calibration PICARRO

In [19]:
picarro_slope = 0
picarro_intercept = 0

df_p.head(1).collect()

# Bottles on 19.12.2023

In [34]:
# bottle 36

# PICARRO
start_date = datetime(2023, 12, 19, 10, 45, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 10, 55, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 607.0610295264346
Cut average: 607.0095882593893
Picarro Average: 0.0


ZeroDivisionError: division by zero